In [4]:
global word, block, rounds
word = 4 #each word has 4 bytes
block = 16 #each block is set to 16 bytes, i.e., 128 bits
rounds = 10 #I am only coding this for 10 rounds in 128-bit AES, but hopefully will have time to expand it

# SubBytes transformation tables and their functions

In [5]:

SubBytes = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)


In [6]:
def subBytes (state):
    for i in range(word):
        for j in range(word):
            state[i][j] = SubBytes[state[i][j]]
    return state
def inverseSubBytes (state):
    for i in range(word):
        for j in range(word):
            state[i][j] = InvSubBytes[state[i][j]]  
    return state


# Other per round functions, excluding the key

ShiftRows

In [7]:
def shiftRows(state):
    print(state)
    state[0][1], state[1][1], state[2][1], state[3][1] = state[1][1], state[2][1], state[3][1], state[0][1]
    state[0][2], state[1][2], state[2][2], state[3][2] = state[2][2], state[3][2], state[0][2], state[1][2]
    state[0][3], state[1][3], state[2][3], state[3][3] = state[3][3], state[0][3], state[1][3], state[2][3]
    #print(state)
            
    return state

def inverseShiftRows(state):
    state[0][1], state[1][1], state[2][1], state[3][1] = state[3][1], state[0][1], state[1][1], state[2][1]
    state[0][2], state[1][2], state[2][2], state[3][2] = state[2][2], state[3][2], state[0][2], state[1][2]
    state[0][3], state[1][3], state[2][3], state[3][3] = state[1][3], state[2][3], state[3][3], state[0][3]
    #print(state)
    return state

In [12]:
def test():
    state = [[[1],[2],[3],[4]],
             [[5],[6],[7],[8]],
             [[9],[10],[11],[12]],
             [[13],[14],[15],[16]]]
    state = shiftRows(state)
    print(state[0][:])
    state = inverseShiftRows(state)
    print(state[0][:])
test()

[[[1], [2], [3], [4]], [[5], [6], [7], [8]], [[9], [10], [11], [12]], [[13], [14], [15], [16]]]
[[1], [6], [11], [16]]
[[1], [2], [3], [4]]


MixColumns! Had trouble with this one, so I referred to other sources to understand. But I am still not managing to make it work with my code as of now.

In [13]:
def mixColumns (col):
    for i in range(word):
        mixColumn(col[i])
    return col

def inverseMixColumns (col):
    for i in range(word):
        a = xtime(xtime(col[i][0] ^ col[i][2]))
        b = xtime(xtime(col[i][1] ^ col[i][3]))
        col[i][0] ^= a
        col[i][1] ^= b
        col[i][2] ^= a
        col[i][3] ^= b
    mixColumns(col)
    return col

def mixColumn(col):
    t = col[0] ^ col[1] ^ col[2] ^ col[3]
    a = col[0]
    col[0] ^= t ^ xtime(col[0] ^ col[1])
    col[1] ^= t ^ xtime(col[1] ^ col[2])
    col[2] ^= t ^ xtime(col[2] ^ col[3])
    col[3] ^= t ^ xtime(col[3] ^ a)
    return col

#taken from the web for mixcolumn XOR operations, couldn't manage to do it on my own due to time
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)


In [14]:
def BytesToMatrix(state):
    shape = []
    for i in range(block):
        byte = (state>>(8*(15-i)))& 0xFF
        if i % word == 0:
            shape.append([byte])
        else:
            shape[i//word].append(byte)
    return shape

def MatrixToBytes(shape):
    state = 0
    for i in range(word):
        for j in range(word):
            state |= (shape[i][j] << (120-8*(4*i+j)))
    return state

In [15]:
def test():
    rando = 4359438891234324234234123189089243251
    state = BytesToMatrix(rando)
    state = mixColumns(state)
    print(state)
    state = inverseMixColumns(state)
    print(state)
test()

[[185, 208, 70, 29], [118, 215, 234, 27], [134, 103, 145, 18], [82, 237, 91, 242]]
[[3, 71, 152, 238], [143, 86, 179, 58], [92, 0, 143, 177], [181, 64, 144, 115]]


# End of AES round functions (minus AddRoundKey)


==================================
# Key operations


In [16]:
RoundConstants = ( 0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36,)

In [17]:
def AddRoundKey(state, key):
    for i in range(word):
        for j in range(word):
            state[i][j] ^= key[i][j] 
    return state

def roundKeys(key):
    r_key = BytesToMatrix(key)
    for i in range(word, word * 11):
        r_key.append([])
        if i % word == 0:
            byte = r_key[i - word][0] ^ SubBytes[r_key[i - 1][1]] ^ RoundConstants[i // word]
            r_key[i].append(byte)
            for j in range(1, word):
                byte = r_key[i - word][j] ^ SubBytes[r_key[i - 1][(j + 1) % word]]
                r_key[i].append(byte)
        else:
            for j in range(word):
                byte = r_key[i - word][j] ^ r_key[i - 1][j]
                r_key[i].append(byte)
    return r_key    

# Encryption

In [18]:
def encrypt (state, key):
    print("Initial: ",state)
    state = BytesToMatrix(state)
    r_key = roundKeys(key)
    state =  AddRoundKey(state, r_key[:word])
    for i in range (rounds):
        print("Round #: ", (i+1))
        print("Input state: ", state)
        roundkey = r_key[word*i:word*(i+1)]
        print("Round key: ", roundkey)
        state = encrypt_round (state, roundkey)
        print("Output state: ", state)      
    state = subBytes(state)
    state = shiftRows(state)
    state = AddRoundKey(state, r_key[40:])
    print(state)
    state = MatrixToBytes(state)
    return state

def encrypt_round (state, key):
    #print("State:",state)
    state = subBytes(state)
    state = shiftRows(state)
    state = mixColumns(state)
    state = AddRoundKey(state, key)
    return state


# Decryption

In [19]:
def decrypt (state, key):
    print("Initial: ",state)
    state = BytesToMatrix(state)
    r_key = roundKeys(key)
    state = AddRoundKey(state, r_key[40:])
    state = inverseShiftRows(state)
    state = inverseSubBytes(state)
    for i in range (rounds):
        print("Round #: ", (i+1))
        print("Input state: ", state)
        roundkey = r_key[word*i:word*(i+1)]
        print("Round key: ", roundkey)
        state = decrypt_round (state, roundkey)
        print("Output state: ", state)         
    state =  AddRoundKey(state, r_key[:word])  
    state = MatrixToBytes(state)
    return state

def decrypt_round (state, key):
    #print("State:",state)
    state = AddRoundKey(state, key)
    state = inverseMixColumns(state)
    state = inverseShiftRows(state)
    state = inverseSubBytes(state)   
    return state

# Test

In [22]:
plaintext = 0x21312343132139803425723498091823
key = 0x31243123aeb231432be2132121324213
cipher = encrypt(plaintext, key)
print("Final CT: ", cipher)


Initial:  44119661605513827902282223338083915811
Round #:  1
Input state:  [[16, 21, 18, 96], [189, 147, 8, 195], [31, 199, 97, 21], [185, 59, 90, 48]]
Round key:  [[49, 36, 49, 35], [174, 178, 49, 67], [43, 226, 19, 33], [33, 50, 66, 19]]
[[202, 89, 201, 208], [122, 220, 48, 46], [192, 198, 239, 89], [86, 226, 190, 4]]
Output state:  [[42, 99, 238, 93], [101, 86, 129, 14], [106, 147, 202, 13], [15, 223, 198, 50]]
Round #:  2
Input state:  [[42, 99, 238, 93], [101, 86, 129, 14], [106, 147, 202, 13], [15, 223, 198, 50]]
Round key:  [[19, 8, 76, 222], [189, 186, 125, 157], [150, 88, 110, 188], [183, 106, 44, 175]]
[[229, 251, 40, 76], [77, 177, 12, 171], [2, 220, 116, 215], [118, 158, 180, 35]]
Output state:  [[93, 43, 149, 105], [160, 223, 75, 186], [168, 174, 68, 65], [150, 50, 219, 119]]
Round #:  3
Input state:  [[93, 43, 149, 105], [160, 223, 75, 186], [168, 174, 68, 65], [150, 50, 219, 119]]
Round key:  [[19, 121, 53, 119], [174, 195, 72, 234], [56, 155, 38, 86], [143, 241, 10, 249

In [23]:
decrypted = decrypt(cipher, key)
print(decrypted)

Initial:  326687893463311547110766383890265502860
Round #:  1
Input state:  [[7, 201, 203, 4], [62, 136, 83, 4], [255, 200, 37, 188], [26, 186, 72, 77]]
Round key:  [[49, 36, 49, 35], [174, 178, 49, 67], [43, 226, 19, 33], [33, 50, 66, 19]]
Output state:  [[252, 1, 47, 220], [241, 23, 225, 195], [141, 12, 208, 107], [196, 102, 185, 36]]
Round #:  2
Input state:  [[252, 1, 47, 220], [241, 23, 225, 195], [141, 12, 208, 107], [196, 102, 185, 36]]
Round key:  [[19, 8, 76, 222], [189, 186, 125, 157], [150, 88, 110, 188], [183, 106, 44, 175]]
Output state:  [[83, 204, 51, 254], [45, 148, 94, 0], [149, 215, 202, 215], [1, 170, 182, 109]]
Round #:  3
Input state:  [[83, 204, 51, 254], [45, 148, 94, 0], [149, 215, 202, 215], [1, 170, 182, 109]]
Round key:  [[19, 121, 53, 119], [174, 195, 72, 234], [56, 155, 38, 86], [143, 241, 10, 249]]
Output state:  [[217, 89, 183, 242], [123, 244, 208, 114], [60, 140, 234, 42], [113, 230, 174, 2]]
Round #:  4
Input state:  [[217, 89, 183, 242], [123, 244, 20

Need to debug the code to match the original PT and decrypted CT, but unfortunately ran out of time due to mid-terms this week and the survey report. I'll work on it for my own understanding after the exam on Thursday.